In [1]:
import numpy as np
import pickle
from pathlib import Path


In [2]:
import sys
from pathlib import Path
PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

In [3]:
from graph import EpilepsyState

In [4]:
import joblib

MODEL_PATH = PROJECT_ROOT / "models" / "lightgbm.pkl"
eeg_model = joblib.load(MODEL_PATH)


In [5]:

# =====================================================
# 2️⃣ MAPPING FUNCTION
# =====================================================

def map_seizure_type(label: int) -> str:
    mapping = {
        0: "healthy",
        1: "generalized",
        2: "focal",
        3: "seizure_event",
    }
    return mapping.get(label, "uncertain")


# =====================================================
# 3️⃣ LOAD EEG DATA
# =====================================================

def load_eeg_features(file_path: str) -> np.ndarray:
    """
    Loads EEG feature vector from comma-separated text.
    """

    with open(file_path, "r") as f:
        text = f.read().strip()

    values = [float(x) for x in text.split(",")]

    features = np.array(values, dtype=np.float32)

    # reshape for model
    return features.reshape(1, -1)


# =====================================================
# 4️⃣ EEG NODE
# =====================================================

def eeg_classifier_node(state: EpilepsyState) -> EpilepsyState:
    """
    EEG classifier node.
    Outputs seizure type only.
    """

    if state.eeg_text_file_path is None:
        state.seizure_type = "uncertain"
        return state

    try:
        # ---- load EEG features ----
        features = load_eeg_features(state.eeg_text_file_path)

        # ---- predict ----
        pred = eeg_model.predict(features)[0]

        # ---- map label ----
        seizure_type = map_seizure_type(pred)

        print("EEG prediction:", pred, seizure_type)

        state.seizure_type = seizure_type

    except Exception as e:
        print("EEG error:", e)
        state.seizure_type = "uncertain"

    return state

In [6]:
if __name__ == "__main__":
    state = EpilepsyState(
        eeg_text_file_path="sample.txt"
    )

    result = eeg_classifier_node(state)
    print("Seizure type:", result.seizure_type)


EEG prediction: 0 healthy
Seizure type: healthy


/home/aasritha/EpilepsyNexus/venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
